<a href="https://colab.research.google.com/github/A01793509/Equipo38/blob/main/MNA_IAyAA_semana_10_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Alberto Jose Garcia Porras (A01793509)
*   Carlos Julio León Caicedo (A01793947)
*   Luis Fernando Ríos Piedra (A00453954)
*   Marco Antonio Vázquez Morales (A01793704)


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [103]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests, zipfile
import seaborn as sns
from io import BytesIO

from sklearn.decomposition import TruncatedSVD,randomized_svd

#**Ejercicio-1.** 

1. Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega 
sus primeros 10 renglones. 

In [104]:
#Extracción de rating_final.csv y geoplaces2.csv desde el sitio fuente

path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
base_datos = path.split('/')[-1]
req_path = requests.get(path)

zipfile.ZipFile(BytesIO(req_path.content)).extractall('/content/sample_data/RCdata')
df_rt = pd.read_csv("/content/sample_data/RCdata/rating_final.csv",sep=',')

path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
base_datos = path.split('/')[-1]
req_path = requests.get(path)

zipfile.ZipFile(BytesIO(req_path.content)).extractall('/content/sample_data/RCdata')
df_g = pd.read_csv("/content/sample_data/RCdata/geoplaces2.csv",sep=',',encoding='latin-1')


In [105]:
#Generar matriz relacional y asignar index con userID
df_rel=pd.merge(df_rt,df_g,how='inner')
df_rel=df_rel.set_index('userID')


In [106]:
df_rel.shape

(1161, 24)

In [107]:
#Generar matriz de utilidad e impresión de 10 renglones
UtMx = df_rel.pivot_table(values='rating', index='userID', columns='name', fill_value=0)

UtMx.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [108]:
#Desplegar tamaño de la matriz
#Tenemos 138 usuarios y 129 restaurantes
UtMx.shape 

(138, 129)

#**Ejercicio-2.**

In [109]:
print("***************************************************")
print('Total de elementos de la matriz de utilidad: %d' % (UtMx.size))
print("***************************************************")
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtMx)))
print("*************************************************")
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtMx) / UtMx.size))
print("*************************************************")

***************************************************
Total de elementos de la matriz de utilidad: 17802
***************************************************
Total de elementos diferentes de cero: 907
*************************************************
Porcentaje de elementos diferentes de cero: 5.1%
*************************************************


In [110]:
X = UtMx.T
X.shape

(129, 138)

In [111]:
Componentes=137
SVDCompleto = TruncatedSVD(n_components=Componentes, random_state=1)  
SVDCompleto.fit(X)

print("******************************************")
print('El total de valores singulares es de: %d' % (SVDCompleto.singular_values_).shape[0])
print("******************************************")

******************************************
El total de valores singulares es de: 129
******************************************


#**Ejercicio-3.**



---

📈 Ejecutaremos el modelo de recomendación utilizando la variable **rating**.


---



In [113]:
Matriz_Resultante = SVDCompleto.fit_transform(X)
corrMx = np.corrcoef(Matriz_Resultante, )

Restaurante_Evaluado = "Subway"

names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)

corr_recomienda = corrMx[id_megusta]

In [114]:
Lista_Restaurantes=list(names[(corr_recomienda > .23) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************")
print("Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************")
for Restaurante in Lista_Restaurantes:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

*********************************************************************************
Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************
1 - Restaurante Chilis Cuernavaca  (Coeficiente Pearson = 0.3011 )
2 - Restaurante Log Yin  (Coeficiente Pearson = 0.2346 )
3 - Restaurante Mariscos Tia Licha  (Coeficiente Pearson = 0.3882 )
4 - Restaurante Mikasa  (Coeficiente Pearson = 0.301 )
5 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.6095 )
6 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.4255 )
7 - Restaurante Restaurant los Pinos  (Coeficiente Pearson = 0.2851 )
8 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.4298 )
9 - Restaurante Sanborns Casa Piedra  (Coeficiente Pearson = 0.4172 )
10 - Restaurante Vips  (Coeficiente Pearson = 0.5063 )


#**Ejercicio-4.**

In [128]:
Componentes=12
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print("*****************************************************")
print('El total de valores singulares encontrados es de: %d' % (SVD.singular_values_).shape[0])

Matriz_Resultante = SVD.fit_transform(X)

corrMx = np.corrcoef(Matriz_Resultante, )
names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)
corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes_RedDim=list(names[(corr_recomienda > .9985) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************************************************")
print("Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************************************************")
for Restaurante in Lista_Restaurantes_RedDim:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')
 


*****************************************************
El total de valores singulares encontrados es de: 12
*********************************************************************************************************************
Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************************************************
1 - Restaurante Log Yin  (Coeficiente Pearson = 0.9993 )
2 - Restaurante Mariscos Tia Licha  (Coeficiente Pearson = 0.9987 )
3 - Restaurante Mikasa  (Coeficiente Pearson = 0.9992 )
4 - Restaurante Paniroles  (Coeficiente Pearson = 0.9987 )
5 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.9996 )
6 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.9996 )
7 - Restaurante Restaurant los Pinos  (Coeficiente Pearson = 0.9996 )
8 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.999

In [129]:
import functools
print("*****************************************************************************")
print("Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:")
print("*****************************************************************************")
print("Se encontraron",len(set(Lista_Restaurantes_RedDim).intersection(Lista_Restaurantes)), "restaurantes iguales.")
print("")
print("************************************************************************************")
print("Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:")
print("************************************************************************************")
[x for x in Lista_Restaurantes + Lista_Restaurantes_RedDim if x not in Lista_Restaurantes or x not in Lista_Restaurantes_RedDim]

*****************************************************************************
Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:
*****************************************************************************
Se encontraron 9 restaurantes iguales.

************************************************************************************
Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:
************************************************************************************


['Chilis Cuernavaca', 'Paniroles']

In [131]:
print("*****************************************************************************")
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % Componentes)
print("*****************************************************************************")
print('%.1f%%' %  (100 * (1- (SVDCompleto.singular_values_[0:Componentes]).sum() / (SVDCompleto.singular_values_).sum())))

*****************************************************************************
Cantidad de información simplificada con los primeros 12 vectores singulares:
*****************************************************************************
72.1%


#**Ejercicio-5.**

In [132]:
Componentes=137
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

Matriz_Resultante = SVD.fit_transform(X)
corrMx = np.corrcoef(Matriz_Resultante, )

Restaurante_Evaluado = "tacos de barbacoa enfrente del Tec"

names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)

corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes=list(names[(corr_recomienda > .17) & (corr_recomienda < 1)])
i=0
print("*************************************************************************************************************")
print("Tomando como referencia el restaurante 'tacos de barbacoa enfrente del Tec' las 10 mejores recomendaciones son:")
print("*************************************************************************************************************")
for Restaurante in Lista_Restaurantes:
  i=i+1
  if i<11:
    Id_Lista = names_list.index(Restaurante)
    print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

*************************************************************************************************************
Tomando como referencia el restaurante 'tacos de barbacoa enfrente del Tec' las 10 mejores recomendaciones son:
*************************************************************************************************************
1 - Restaurante Carreton de Flautas y Migadas  (Coeficiente Pearson = 0.4078 )
2 - Restaurante Hamburguesas La perica  (Coeficiente Pearson = 0.1911 )
3 - Restaurante Little Cesarz  (Coeficiente Pearson = 0.2585 )
4 - Restaurante Taqueria EL amigo   (Coeficiente Pearson = 0.4372 )
5 - Restaurante carnitas mata calle Emilio Portes Gil  (Coeficiente Pearson = 0.1709 )
6 - Restaurante carnitas_mata  (Coeficiente Pearson = 0.1709 )
7 - Restaurante little pizza Emilio Portes Gil  (Coeficiente Pearson = 0.6538 )
8 - Restaurante palomo tec  (Coeficiente Pearson = 0.1952 )
9 - Restaurante puesto de gorditas  (Coeficiente Pearson = 0.4128 )
10 - Restaurante tacos abi  

In [133]:
Componentes=10
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print("****************************************************")
print('El total de valores singulares encontrados es de: %d' % (SVD.singular_values_).shape[0])

Matriz_Resultante = SVD.fit_transform(X)

corrMx = np.corrcoef(Matriz_Resultante, )

corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes_RedDim=list(names[(corr_recomienda > .998) & (corr_recomienda < 1)])
i=0
print("**************************************************************************************************************************************************")
print("Reduciendo la dimensionalidad y manteniendo como referencia el restaurante 'tacos de barbacoa enfrente del Tec' las 10 mejores recomendaciones son:")
print("**************************************************************************************************************************************************")
for Restaurante in Lista_Restaurantes_RedDim:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

****************************************************
El total de valores singulares encontrados es de: 10
**************************************************************************************************************************************************
Reduciendo la dimensionalidad y manteniendo como referencia el restaurante 'tacos de barbacoa enfrente del Tec' las 10 mejores recomendaciones son:
**************************************************************************************************************************************************
1 - Restaurante Hamburguesas La perica  (Coeficiente Pearson = 0.9983 )
2 - Restaurante Little Cesarz  (Coeficiente Pearson = 0.9982 )
3 - Restaurante Pollo_Frito_Buenos_Aires  (Coeficiente Pearson = 0.9995 )
4 - Restaurante Taqueria EL amigo   (Coeficiente Pearson = 0.9999 )
5 - Restaurante carnitas_mata  (Coeficiente Pearson = 0.9994 )
6 - Restaurante little pizza Emilio Portes Gil  (Coeficiente Pearson = 0.9997 )
7 - Restaurante palomo tec  (Coe

In [134]:
import functools
print("*****************************************************************************")
print("Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:")
print("*****************************************************************************")
print("Se encontraron",len(set(Lista_Restaurantes_RedDim).intersection(Lista_Restaurantes)), "restaurantes iguales.")
print("")
print("************************************************************************************")
print("Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:")
print("************************************************************************************")
[x for x in Lista_Restaurantes + Lista_Restaurantes_RedDim if x not in Lista_Restaurantes or x not in Lista_Restaurantes_RedDim]

*****************************************************************************
Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:
*****************************************************************************
Se encontraron 9 restaurantes iguales.

************************************************************************************
Los siguientes restaurantes difieren en la recomendación al ajustar dimensionalidad:
************************************************************************************


['Carreton de Flautas y Migadas',
 'carnitas mata calle Emilio Portes Gil',
 'Pollo_Frito_Buenos_Aires']

In [135]:
print("*****************************************************************************")
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % Componentes)
print("*****************************************************************************")
print('%.1f%%' %  (100 * (1- (SVDCompleto.singular_values_[0:Componentes]).sum() / (SVDCompleto.singular_values_).sum())))

*****************************************************************************
Cantidad de información simplificada con los primeros 10 vectores singulares:
*****************************************************************************
75.6%




---

✍ **Conclusiones:**


---



#**Ejercicio-6.**



---

📈 Ejecutaremos el modelo de recomendación utilizando la variable **food_rating**.


---



In [52]:
#Generar matriz relacional y asignar index con userID
df_rel=pd.merge(df_rt,df_g,how='inner')
df_rel=df_rel.set_index('userID')
df_rel.shape

#Generar matriz de utilidad e impresión de 10 renglones
UtMx = df_rel.pivot_table(values='food_rating', index='userID', columns='name', fill_value=0)

UtMx.head(10)
UtMx.shape 

(138, 129)

In [141]:
print("***************************************************")
print('Total de elementos de la matriz de utilidad: %d' % (UtMx.size))
print("***************************************************")
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtMx)))
print("*************************************************")
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtMx) / UtMx.size))
print("*************************************************")

X = UtMx.T
X.shape

Componentes=137
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)
print('El total de valores singulares es de: %d' % (SVD.singular_values_).shape[0])
print("******************************************")

***************************************************
Total de elementos de la matriz de utilidad: 17802
***************************************************
Total de elementos diferentes de cero: 907
*************************************************
Porcentaje de elementos diferentes de cero: 5.1%
*************************************************
El total de valores singulares es de: 129
******************************************


In [142]:
Matriz_Resultante = SVD.fit_transform(X)
corrMx = np.corrcoef(Matriz_Resultante, )

Restaurante_Evaluado = "Subway"

names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)

corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes=list(names[(corr_recomienda > .232) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************")
print("Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************")
for Restaurante in Lista_Restaurantes:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

*********************************************************************************
Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************
1 - Restaurante Chilis Cuernavaca  (Coeficiente Pearson = 0.3011 )
2 - Restaurante Log Yin  (Coeficiente Pearson = 0.2346 )
3 - Restaurante Mariscos Tia Licha  (Coeficiente Pearson = 0.3882 )
4 - Restaurante Mikasa  (Coeficiente Pearson = 0.301 )
5 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.6095 )
6 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.4255 )
7 - Restaurante Restaurant los Pinos  (Coeficiente Pearson = 0.2851 )
8 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.4298 )
9 - Restaurante Sanborns Casa Piedra  (Coeficiente Pearson = 0.4172 )
10 - Restaurante Vips  (Coeficiente Pearson = 0.5063 )


In [148]:
Componentes=7
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print("****************************************************")
print('El total de valores singulares encontrados es de: %d' % (SVD.singular_values_).shape[0])

Matriz_Resultante = SVD.fit_transform(X)

corrMx = np.corrcoef(Matriz_Resultante, )
corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes_RedDim=list(names[(corr_recomienda > .9999875) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************************************************")
print("Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************************************************")
for Restaurante in Lista_Restaurantes_RedDim:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],6),')')

****************************************************
El total de valores singulares encontrados es de: 7
*********************************************************************************************************************
Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************************************************
1 - Restaurante Log Yin  (Coeficiente Pearson = 0.999998 )
2 - Restaurante Paniroles  (Coeficiente Pearson = 0.999995 )
3 - Restaurante Restaurant Bar Coty y Pablo  (Coeficiente Pearson = 0.999988 )
4 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.999998 )
5 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.999994 )
6 - Restaurante Restaurant los Pinos  (Coeficiente Pearson = 0.999989 )
7 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.999999 )
8 - Restaurante Sanborns Casa P

In [149]:
print("*****************************************************************************")
print("Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:")
print("*****************************************************************************")
print("Se encontraron",len(set(Lista_Restaurantes_RedDim).intersection(Lista_Restaurantes)), "restaurantes iguales.")
print("")

print("*****************************************************************************")
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % Componentes)
print("*****************************************************************************")
print('%.1f%%' %  (100 * (1- (SVDCompleto.singular_values_[0:Componentes]).sum() / (SVDCompleto.singular_values_).sum())))

*****************************************************************************
Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:
*****************************************************************************
Se encontraron 7 restaurantes iguales.

*****************************************************************************
Cantidad de información simplificada con los primeros 7 vectores singulares:
*****************************************************************************
81.3%




---

✍ **Conclusiones:** Reduciendo el número de componentes, el máximo de coincidencias encontrada fue de 6.


---



#**Ejercicio-7.**



---

📈 Ejecutaremos el modelo de recomendación utilizando la variable **service_rating**.


---



In [150]:
#Generar matriz relacional y asignar index con userID
df_rel=pd.merge(df_rt,df_g,how='inner')
df_rel=df_rel.set_index('userID')
df_rel.shape

#Generar matriz de utilidad e impresión de 10 renglones
UtMx = df_rel.pivot_table(values='service_rating', index='userID', columns='name', fill_value=0)

UtMx.head(10)
UtMx.shape 

(138, 129)

In [153]:
print("***************************************************")
print('Total de elementos de la matriz de utilidad: %d' % (UtMx.size))
print("***************************************************")
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtMx)))
print("*************************************************")
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtMx) / UtMx.size))
print("*************************************************")
X = UtMx.T
X.shape

Componentes=137
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print('El total de valores singulares es de: %d' % (SVD.singular_values_).shape[0])

***************************************************
Total de elementos de la matriz de utilidad: 17802
***************************************************
Total de elementos diferentes de cero: 846
*************************************************
Porcentaje de elementos diferentes de cero: 4.8%
*************************************************
El total de valores singulares es de: 129


In [154]:
Matriz_Resultante = SVD.fit_transform(X)
corrMx = np.corrcoef(Matriz_Resultante, )

Restaurante_Evaluado = "Subway"

names = UtMx.columns
names_list = list(names)
id_megusta = names_list.index(Restaurante_Evaluado)

corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes=list(names[(corr_recomienda > .17) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************")
print("Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************")
for Restaurante in Lista_Restaurantes:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

*********************************************************************************
Tomando como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************
1 - Restaurante Chilis Cuernavaca  (Coeficiente Pearson = 0.2307 )
2 - Restaurante Giovannis  (Coeficiente Pearson = 0.1928 )
3 - Restaurante Kiku Cuernavaca  (Coeficiente Pearson = 0.1787 )
4 - Restaurante Paniroles  (Coeficiente Pearson = 0.3659 )
5 - Restaurante Restaurant Familiar El Chino  (Coeficiente Pearson = 0.1829 )
6 - Restaurante Restaurant Las Mananitas  (Coeficiente Pearson = 0.2263 )
7 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.773 )
8 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.6002 )
9 - Restaurante Rincon del Bife  (Coeficiente Pearson = 0.5187 )
10 - Restaurante Vips  (Coeficiente Pearson = 0.4212 )


In [155]:
Componentes=15
SVD = TruncatedSVD(n_components=Componentes, random_state=1)  
SVD.fit(X)

print("****************************************************")
print('El total de valores singulares encontrados es de: %d' % (SVD.singular_values_).shape[0])

Matriz_Resultante = SVD.fit_transform(X)

corrMx = np.corrcoef(Matriz_Resultante, )
corr_recomienda = corrMx[id_megusta]

Lista_Restaurantes_RedDim=list(names[(corr_recomienda > .81) & (corr_recomienda < 1)])
i=0
print("*********************************************************************************************************************")
print("Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:")
print("*********************************************************************************************************************")
for Restaurante in Lista_Restaurantes_RedDim:
  i=i+1
  Id_Lista = names_list.index(Restaurante)
  print(i,'- Restaurante',Restaurante, ' (Coeficiente Pearson =', round(corr_recomienda[Id_Lista],4),')')

****************************************************
El total de valores singulares encontrados es de: 15
*********************************************************************************************************************
Reduciendo la dimensionalidad y manteniendo como referencia el restaurante Subway las 10 mejores recomendaciones son:
*********************************************************************************************************************
1 - Restaurante Chilis Cuernavaca  (Coeficiente Pearson = 0.924 )
2 - Restaurante El Oceano Dorado  (Coeficiente Pearson = 0.8136 )
3 - Restaurante El cotorreo  (Coeficiente Pearson = 0.9007 )
4 - Restaurante Paniroles  (Coeficiente Pearson = 0.9028 )
5 - Restaurante Restaurant Teely  (Coeficiente Pearson = 0.8904 )
6 - Restaurante Restaurant and Bar and Clothesline Carlos N Charlies  (Coeficiente Pearson = 0.899 )
7 - Restaurante Restaurant los Pinos  (Coeficiente Pearson = 0.8383 )
8 - Restaurante Rincon del Bife  (Coeficiente Pears

In [157]:
print("*****************************************************************************")
print("Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:")
print("*****************************************************************************")
print("Se encontraron",len(set(Lista_Restaurantes_RedDim).intersection(Lista_Restaurantes)), "restaurantes iguales.")
print("")
print("*****************************************************************************")
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % Componentes)
print("*****************************************************************************")
print('%.1f%%' %  (100 * (1- (SVDCompleto.singular_values_[0:Componentes]).sum() / (SVDCompleto.singular_values_).sum())))

*****************************************************************************
Coincidencias en recomendación de restaurantes al reducir la dimensionalidad:
*****************************************************************************
Se encontraron 6 restaurantes iguales.

*****************************************************************************
Cantidad de información simplificada con los primeros 15 vectores singulares:
*****************************************************************************
67.1%




---

✍ **Conclusiones:** Reduciendo el número de componentes, el máximo de coincidencias encontrada fue de 6.


---



#**Ejercicio-8.**



---

✍ **Conclusiones Finales:** Incluye tus conclusiones finales. En particular indica la coincidencia o no de estos tres modelos.


---



###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**